In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import sounddevice as sd
import plotly.express as px
from rich import print
import math
from plotly.subplots import make_subplots

In [ ]:
def fft_and_waveform(samples, fs=40_000.0):
    N = len(samples)
    # if N != 1024:
    #     raise ValueError("Expected 1024 samples")

    # Convert to numpy array
    x = np.asarray(samples, dtype=np.float64)

    # Time axis
    t = np.arange(N) / fs

    # Windowed FFT
    w = np.hanning(N)
    xw = (x - x.mean()) * w
    X = np.fft.rfft(xw, n=N)
    freqs = np.fft.rfftfreq(N, d=1/fs)

    cg = w.mean()
    mag = np.abs(X) / (N * cg)

    # Two subplots: waveform (top), FFT (bottom)
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Waveform", "FFT Spectrum"),
        vertical_spacing=0.15
    )

    # Time-domain waveform
    fig.add_trace(go.Scatter(
        x=t, y=x, mode="lines", name="Waveform"
    ), row=1, col=1)

    # Frequency-domain spectrum
    fig.add_trace(go.Scatter(
        x=freqs, y=mag, mode="lines", name="Magnitude Spectrum"
    ), row=2, col=1)

    fig.update_xaxes(title="Time (s)", row=1, col=1)
    fig.update_yaxes(title="Amplitude", row=1, col=1)

    fig.update_xaxes(title="Frequency (Hz)", row=2, col=1)
    fig.update_yaxes(title="Magnitude", row=2, col=1)

    fig.update_layout(height=600, template="plotly_white")

    return fig


In [ ]:
def plot_spectrogram(samples, fs=10_000.0, n_fft=1024, hop=512, max_freq=None, min_freq=None):
    x = np.asarray(samples, dtype=np.float64)
    N = x.size

    # Hann window
    w = np.hanning(n_fft)

    # Number of frames
    n_frames = 1 + (N - n_fft) // hop
    n_bins = n_fft // 2 + 1

    # STFT
    S = np.empty((n_bins, n_frames), dtype=np.complex128)
    for i in range(n_frames):
        start = i * hop
        frame = x[start:start + n_fft]
        frame = (frame - frame.mean()) * w
        S[:, i] = np.fft.rfft(frame, n=n_fft)

    # Magnitude → dB
    mag = np.abs(S)
    mag /= (n_fft * (w.mean() if w.mean() != 0 else 1.0))
    S_db = 20.0 * np.log10(np.maximum(mag, 1e-12))

    # Axes
    times = (np.arange(n_frames) * hop) / fs
    freqs = np.fft.rfftfreq(n_fft, d=1.0/fs)

    # Apply frequency mask
    if max_freq is not None:
        mask = freqs <= max_freq
        freqs = freqs[mask]
        S_db = S_db[mask, :]

    if min_freq is not None:
        mask = freqs >= min_freq
        freqs = freqs[mask]
        S_db = S_db[mask, :]

    # Plot
    plt.figure(figsize=(9, 5))
    plt.imshow(
        S_db,
        origin='lower',
        aspect='auto',
        extent=[times[0], times[-1], freqs[0], freqs[-1]]
    )
    plt.colorbar(label='Magnitude (dB)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title(f'Spectrogram up to {max_freq or fs/2:g} Hz')
    plt.tight_layout()
    plt.show()
    return S_db


In [ ]:
def play_array(samples, fs=40_000):
    # Convert to float32
    x = np.asarray(samples, dtype=np.float32)

    # If it's raw ADC (unsigned, centered at ~2048), shift and scale
    if x.min() >= 0:
        x = (x - np.mean(x)) / np.max(np.abs(x))

    # Play
    sd.play(x, samplerate=fs)
    sd.wait()

In [ ]:
# Return True if n is a power of two and >= 2, else False
def is_pow2(n: int) -> bool:
    return (n >= 2) and ((n & (n - 1)) == 0)

# In-place bit-reversal permutation on re/im lists
def bit_reverse(re: list, im: list, n: int) -> None:
    j = 0
    for i in range(1, n - 1):
        bit = n >> 1
        while j & bit:
            j ^= bit
            bit >>= 1
        j ^= bit
        if i < j:
            re[i], re[j] = re[j], re[i]
            im[i], im[j] = im[j], im[i]

# In-place complex FFT (inverse = 0 forward, 1 inverse). Returns 0 on success, -1 on bad n.
def fft_simple(re: list, im: list, n: int, inverse: int) -> int:
    if not is_pow2(n):
        return -1

    # Put data in bit-reversed order
    bit_reverse(re, im, n)

    # Cooley–Tukey stages
    len_ = 2
    while len_ <= n:
        ang = (+1.0 if inverse else -1.0) * (2.0 * math.pi / float(len_))
        i = 0
        half = len_ >> 1
        while i < n:
            j = 0
            while j < half:
                theta = ang * float(j)
                w_re = math.cos(theta)
                w_im = math.sin(theta)

                p = i + j
                q = p + half

                ur = re[p]
                ui = im[p]
                tr = re[q] * w_re - im[q] * w_im
                ti = re[q] * w_im + im[q] * w_re

                re[p] = ur + tr
                im[p] = ui + ti
                re[q] = ur - tr
                im[q] = ui - ti

                j += 1
            i += len_
        len_ <<= 1

    # Normalize for inverse transform
    if inverse:
        invn = 1.0 / float(n)
        for k in range(n):
            re[k] *= invn
            im[k] *= invn

    return 0

# Convenience wrapper for real input: copies x -> re, zeros im, then forward FFT
def fft_real_simple(x: list) -> tuple[list, list]:
    n = len(x)
    if not is_pow2(n):
        raise ValueError("Length must be a power of two (>= 2)")
    re = [float(v) for v in x]
    im = [0.0] * n
    err = fft_simple(re, im, n, inverse=0)
    if err != 0:
        raise ValueError("fft_simple failed")
    return re, im

# Optional: magnitude spectrum |X[k]|
def fft_mag(re: list, im: list) -> list:
    return [math.hypot(re[k], im[k]) for k in range(len(re))]



In [ ]:
def one_hz_sliding(x, fs, f0, n_fft=1024, window='hann', scale_like_fft=True):
    """
    Slide a length-n_fft window by 1 sample over x and compute the magnitude at frequency f0.
    - x: list/array of samples
    - fs: sample rate (Hz)
    - f0: target frequency (Hz)  (use f0=1.0 if you literally mean 1 Hz)
    - n_fft: window size (e.g., 1024)
    - window: 'hann' or None
    - scale_like_fft: if True, scales similar to |FFT bin| (≈ 2/(N*CG))

    Returns:
      times: list of center times (seconds) for each window
      mags:  list of magnitudes (linear)
    """
    N = n_fft
    H = 1  # slide by one sample
    L = len(x)
    if L < N:
        raise ValueError("Signal shorter than window.")

    # Window + coherent gain
    if window == 'hann':
        w = [0.5 - 0.5 * math.cos(2.0 * math.pi * n / (N - 1)) for n in range(N)]
        cg = sum(w) / N  # ~0.5 for Hann
    else:
        w = [1.0] * N
        cg = 1.0

    # Goertzel coefficient
    w0 = 2.0 * math.pi * (f0 / fs)
    c = 2.0 * math.cos(w0)

    frames = L - N + 1
    mags = [0.0] * frames
    times = [ (i + N/2.0) / fs for i in range(frames) ]

    # Brute-force per window (simple, readable)
    for i in range(frames):
        s0 = 0.0
        s1 = 0.0
        # windowed segment
        for n in range(N):
            xn = x[i + n] * w[n]
            s2 = s1
            s1 = s0
            s0 = xn + c * s1 - s2

        # Goertzel single-bin magnitude
        mag2 = s0*s0 + s1*s1 - c*s0*s1
        mag = math.sqrt(mag2)

        if scale_like_fft:
            mag *= (2.0 / (N * cg))

        mags[i] = mag

    return times, mags


In [ ]:
# Make a 1024-sample 440 Hz sine at 40 kHz
fs = 256
N = 1024*1
x = [
    math.sin(2.0 * math.pi * 4   * (n + 2) / N ) + 
     math.sin(2.0 * math.pi * 3.0  * (n + 3) / N )
     + math.sin(2.0 * math.pi * (20 + 0)  * (n + 3) / N ) 
     for n in range(N)]

fs = N
f0 = 20          # set to 1.0 if you literally want 1 Hz
t, a = one_hz_sliding(x, fs, f0, n_fft=1024, window='hann')
px.line(a).show()

# Forward FFT (real input)
re, im = fft_real_simple(x)

# Magnitude spectrum
mag = fft_mag(re, im)

# re, im now hold X[k]; mag is the linear amplitude per bin
px.line(x).show()
px.line(mag[:len(mag)//2]).show()


In [ ]:
arr = pd.read_excel('sh.xlsx', sheet_name='Sheet2').iloc[:1024,0].tolist()
fig = fft_and_waveform(arr)
fig.show()

In [ ]:
arr = pd.read_excel('sh.xlsx', sheet_name='Sheet2').iloc[:,0].to_numpy()

In [ ]:
info = goertzel_sliding(arr, 40000, 200)
px.line(arr).show()

In [ ]:
play_array(arr, 10000)
info = plot_spectrogram(arr, max_freq=1600, hop=128, min_freq=140)

In [ ]:
16383, 0x3FFF

In [ ]:
fft_and_waveform([i if i > -5 else i for i in info[0,:]], fs=618/4)

In [ ]:
np.pi/2

In [ ]:
from pathlib import Path

Path("sine_table.h").write_text('const float SIN[65536] = {\n'
    + ',\n'.join([str(i+1) for i in np.sin(np.linspace(0, np.pi*2, 0x10000))])
    + '\n};\n')


In [ ]:

Path("exp_table.h").write_text('const float EXP[1000] = {\n'
    + 'f,\n'.join([str(round(i, 8)) for i in np.exp(np.linspace(10, -10, 1000))])
    + '\n};\n')

In [ ]:


87*69/ (60*72) 
1750/25.4 / 12
.75*12
255 / .5
351.5376 +354.5487+ 424.3042+ 428.8208+ 488.2884,347.5229 +458.9310+ 396.7031+ 438.8575+ 405.4851
255/256/250

from math import cos, sin, asin, acos, pi
(cos(1.001) - cos(1))*1000, -sin(1), asin(0.8414709848078965)
# 0.0000019432
# 3.11
# 0.0004632713
((1-cos(3.11)) - (1-cos(3.11+.5*pi*.001)))*pi*pi/255, asin(0.950000194321)
.077408/255/3.14159 * 2
32*pi*.091

In [ ]:
#Parabola Generator
import math

def generate_paraboloid(focal_length, diameter, min_thickness, output_file):
    radius = diameter / 2.0
    f = focal_length

    # Adjust mesh density (more segments = finer detail)
    radial_segments = 200  # around the circle
    depth_segments = 200   # along the parabola

    vertices = []
    faces = []

    # Generate parabola surface
    for i in range(depth_segments + 1):
        r = radius * (i / depth_segments)
        z = -(r * r) / (4.0 * f)

        for j in range(radial_segments):
            theta = 2.0 * math.pi * (j / radial_segments)
            x = r * math.cos(theta)
            y = r * math.sin(theta)
            vertices.append((x, y, z))

    # Backplane (flat surface at max depth + thickness)
    max_depth = (radius * radius) / (4.0 * f)
    back_z = max_depth + min_thickness
    for i in range(depth_segments + 1):
        r = radius * (i / depth_segments)
        for j in range(radial_segments):
            theta = 2.0 * math.pi * (j / radial_segments)
            x = r * math.cos(theta)
            y = r * math.sin(theta)
            vertices.append((x, y, back_z))

    # Build faces (parabola surface)
    for i in range(depth_segments):
        for j in range(radial_segments):
            p0 = i * radial_segments + j
            p1 = i * radial_segments + (j + 1) % radial_segments
            p2 = (i + 1) * radial_segments + (j + 1) % radial_segments
            p3 = (i + 1) * radial_segments + j
            faces.append((p0, p1, p2, p3))

    # Build faces (backplane)
    offset = (depth_segments + 1) * radial_segments
    for i in range(depth_segments):
        for j in range(radial_segments):
            p0 = offset + i * radial_segments + j
            p1 = offset + i * radial_segments + (j + 1) % radial_segments
            p2 = offset + (i + 1) * radial_segments + (j + 1) % radial_segments
            p3 = offset + (i + 1) * radial_segments + j
            faces.append((p0, p1, p2, p3))

    # Connect parabola rim to backplane rim
    for j in range(radial_segments):
        p0 = depth_segments * radial_segments + j
        p1 = depth_segments * radial_segments + (j + 1) % radial_segments
        p2 = offset + depth_segments * radial_segments + (j + 1) % radial_segments
        p3 = offset + depth_segments * radial_segments + j
        faces.append((p0, p1, p2, p3))

    # Write OBJ file
    with open(output_file, "w") as f:
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        for face in faces:
            f.write(f"f {' '.join(str(idx+1) for idx in face)}\n")

    print(f"OBJ file written: {output_file}")
    print(f"Vertices: {len(vertices)}, Faces: {len(faces)}")

generate_paraboloid(1750, 255, 10, 'parabola.obj')

In [37]:
from pathlib import Path
import numpy as np, plotly.express as px
h = 500

labels = {0:'angle', 1:'info->rotate_angle_step', 2:'xamp', 3:'info->xamp_step'}
ar = np.array([[float(j) for j in i.split()] for i in Path('t').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])
for i in range(ar.shape[1]):
    px.line(ar[:,i], width=1000, height=h, title=labels[i]).show()
    # ar2 = np.diff(ar[:,i])
    # px.line(ar2[(ar2>-2) & (ar2<39)], width=1000, height=h).show()


In [25]:
data_string = """0.005463 2.609804 954 
0.003690 2.513726 1077 
0.004537 2.637255 962 
0.003973 2.245098 926 
0.006686 2.566667 676 
0.005651 2.423529 733 
0.006106 2.501961 695 
0.006953 2.556863 623 
0.003675 2.556863 1182 
0.006875 2.356863 616 
0.006169 2.400000 689 
0.003220 2.696079 1405 
0.006278 2.229412 575 
0.003361 2.533333 1209 
0.006263 2.417647 649 
0.006325 2.439216 652 
0.005259 2.241177 680 
0.005902 2.335294 666 
0.006827 2.411765 625 
0.005604 2.331372 711 
0.003878 2.215686 953 
0.006184 2.513726 738 
0.005588 2.503922 784 
0.003643 2.431373 1042 
0.003518 2.643137 1199 
0.003235 2.445098 1146 
0.004569 2.205882 719 
0.006827 2.260784 544 
0.004239 2.631373 1024 
0.005369 2.407843 716 
0.006545 2.674510 677 
0.005620 2.582353 754 
0.004757 2.627451 909 
0.003125 2.537255 1266 
0.006718 2.354902 582 
0.003408 2.496078 1227 
0.004427 2.464706 954 
0.005102 2.513726 862 
0.005635 2.596078 856 
0.004882 2.366667 885 
0.006780 2.650980 676 
0.006153 2.572549 695 
0.006122 2.339216 653 
0.004537 2.405882 894 
0.006969 2.211765 549 
0.005588 2.617647 838 
0.003925 2.654902 1136 
0.003816 2.423529 998 
0.006561 2.241177 543 
0.006592 2.513726 622 
0.003690 2.613726 1195 
0.005792 2.645098 795 
0.006310 2.680392 703 
0.004882 2.543137 832 
0.004427 2.366667 880 
0.005729 2.584314 782 
0.004176 2.388235 932 
0.005055 2.649020 878 
0.006608 2.456863 669 
0.005667 2.254902 707 
0.005275 2.307843 739 
0.004427 2.672549 1058 
0.006655 2.264706 583 
0.004255 2.682353 1031 
0.006812 2.400000 567 
0.003110 2.392157 1145 
0.003141 2.313725 1104 
0.003816 2.617647 1087 
0.004396 2.470588 885 
0.005808 2.229412 654 
0.006325 2.335294 644 
0.003925 2.421569 1041 
0.006043 2.223530 625 
0.006090 2.501961 718 
0.003361 2.335294 1191 
0.005541 2.511765 755 
0.003078 2.662745 1369 
0.006796 2.547059 641 
0.006953 2.227451 540 
0.003784 2.425490 1045 
0.005431 2.492157 787 
0.004176 2.223530 861 
0.004349 2.260784 810 
0.006169 2.298039 603 
0.003031 2.362745 1173 
0.005306 2.574510 827 
0.005447 2.294118 728 
0.004882 2.645098 966 
0.004522 2.643137 1054 
0.006404 2.396079 655 
0.003329 2.668628 1302 
0.003329 2.360784 1115 
0.005227 2.301961 770 
0.004380 2.276471 884 
0.003518 2.464706 1110 
0.004631 2.341177 784 
0.004082 2.403922 900 
0.003502 2.239216 1011 
0.004647 2.205882 775 
0.006514 2.621569 664 
0.004678 2.519608 887 
0.003471 2.666667 1300 
0.003204 2.607843 1392 
0.006169 2.311765 645 
0.005400 2.284314 689 
0.004867 2.637255 926 
0.006012 2.462745 690 
0.006686 2.392157 604 
0.006671 2.213726 588 
0.005290 2.545098 871 
0.005416 2.350981 754 
0.005337 2.601961 804 
0.004553 2.466667 850 
0.004051 2.678432 1062 
0.004678 2.329412 861 
0.004176 2.403922 1058 
0.005714 2.594118 785 
0.005180 2.505883 821 
0.003455 2.686275 1306 
0.006231 2.688235 727 
0.006859 2.486274 640 
0.003643 2.588235 1207 
0.006984 2.568628 618 
0.003910 2.290196 902 
0.004365 2.335294 815 
0.006545 2.252941 587 
0.006529 2.545098 687 
0.003439 2.552941 1232 
0.005494 2.696079 864 
0.005682 2.639216 817 
0.006090 2.317647 643 
0.006451 2.321569 618 
0.006200 2.643137 745 
0.006341 2.292157 606 
0.006780 2.643137 690 
0.005855 2.515686 740 
0.004804 2.623529 932 
0.005667 2.541177 753 
0.003784 2.672549 1231 
0.005322 2.600000 885 
0.004725 2.549020 893 
0.006341 2.547059 655 
0.005086 2.386275 743 
0.003314 2.700000 1307 
0.005369 2.535294 810 
0.006702 2.219608 555 
0.006576 2.541177 658 
0.004035 2.227451 880   """

In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def fit_surface_and_plot(df, degree=2, grid_size=30):
    # --- Fit surface ---
    X = df[['x_data', 'y_data']].values
    y = df['z_data'].values

    poly = PolynomialFeatures(degree=degree, include_bias=True)
    X_poly = poly.fit_transform(X)

    model = LinearRegression()
    model.fit(X_poly, y)

    # Get coefficients for pretty print
    coefs = dict(zip(poly.get_feature_names_out(['x','y']), model.coef_))
    coefs['intercept'] = model.intercept_

    # Print the explicit equation
    terms = [f"{coef:+.4f}*{name}" for name, coef in coefs.items() if name not in ('1','intercept')]
    formula = f"z = {coefs['intercept']:.4f} " + " ".join(terms)
    print("Best-fit polynomial surface:")
    print(formula)

    # --- Predict over a grid for plotting ---
    x_range = np.linspace(df['x_data'].min(), df['x_data'].max(), grid_size)
    y_range = np.linspace(df['y_data'].min(), df['y_data'].max(), grid_size)
    xx, yy = np.meshgrid(x_range, y_range)

    X_grid = np.column_stack([xx.ravel(), yy.ravel()])
    zz = model.predict(poly.transform(X_grid)).reshape(xx.shape)

    # --- Plot training data + surface ---
    fig = go.Figure()

    # Training data as scatter points
    fig.add_trace(go.Scatter3d(
        x=df['x_data'], y=df['y_data'], z=df['z_data'],
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Training Data'
    ))

    # Fitted surface
    fig.add_trace(go.Surface(
        x=xx, y=yy, z=zz,
        colorscale='Viridis',
        opacity=0.6,
        name='Fitted Surface'
    ))

    fig.update_layout(
        scene=dict(
            xaxis_title='x_data',
            yaxis_title='y_data',
            zaxis_title='z_data'
        ),
        title=f"Polynomial Surface Fit (degree={degree})"
    )

    fig.show()

    # Return the model + poly for reuse
    return model, poly

# Example usage:
# df = pd.read_csv(io.StringIO(data_string), sep=' ', header=None)
# df.columns = ['x_data', 'y_data', 'z_data']
# model, poly = fit_surface_and_plot(df)


In [26]:
import pandas as pd
import io
import plotly.express as px

data_string = '\n'.join([i.strip() for i in data_string.splitlines()])

# Use io.StringIO to read the string as if it were a file
df = pd.read_csv(io.StringIO(data_string), sep=' ', header=None)
df.columns = ['x_data', 'y_data', 'z_data']
model, poly = fit_surface_and_plot(df)


# Create a 3D scatter plot with color and hover information
fig = px.scatter_3d(df, x='x_data', y='y_data', z='z_data',
                    color='z_data',  # Use the Z value to determine marker color
                    size='z_data',   # Use the Z value to determine marker size (optional, for emphasis)
                    title='3D Scatter Plot: X, Y, and Z Correlation',
                    labels={'x_data': 'X Value', 'y_data': 'Y Value', 'z_data': 'Z Value'},
                    hover_data=['x_data', 'y_data', 'z_data'],
                    color_continuous_scale=px.colors.sequential.Viridis,
                    height=800)

# Add custom layout features for better clarity
fig.update_layout(
    scene=dict(
        xaxis_title='X Value',
        yaxis_title='Y Value',
        zaxis_title='Z Value'
    )
)

# Display the interactive plot
fig.show()


Best-fit polynomial surface:
z = -146.8796 -141950.2655*x +964.1840*y +25247540.5481*x^2 -110667.1081*x y +7.8001*y^2
